<img style="float: right; width: 150px;" src="https://raw.githubusercontent.com/firrm/DAI/main/assets/firrm.jpg">

## <span style="color:#4375c7">Quantitative Finance</span>
***
*Course materials are for educational purposes only. Nothing contained herein should be considered investment advice or an opinion regarding the suitability of any security. For more information about this course, please contact us.*
***


## Introduction to the Scientific Toolbox of Python: SciPy
In addition to NumPy, we have another open source library, **SciPy**. This library contains algorithms and mathematical tools for manipulating NumPy objects with very specific scientific and engineering goals. It has been the environment of choice for many applied mathematicians for years. SciPy adds significant power to Python for statistical engineering and visualization. For more information about the library, see the **[SciPy documentation](https://docs.scipy.org/doc/scipy/reference/#)**.



### Session contents:
1. **[Notebook set up](#setup)**
2. **[Basic statistics in SciPy](#basic)**
3. **[Optimization with SciPy](#optimization)** 
4. **[Session Takeaways](#takeaway)**
***

## 1. Notebook set up  <a name="setup"></a> 
To begin, import Pandas, NumPy, Matplotlib, SciPy, and WRDS.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy 
import wrds

Let us now import some data from the **[Wharton Research Data Services (WRDS)](https://wrds-www.wharton.upenn.edu/)**. To do this, connect to the WRDS database using `wrds.Connection()` and enter your credentials, respectively.

In [ ]:
db = wrds.Connection()

Load some data from Wharton using the `.get_table()` method of your connected database `db`. Specify the input parameters in your query function and store your DataFrame in `data`. To check the input parameters, see `help(db.get_table)`. More information about Python queries with WRDS can be found **[here](https://wrds-www.wharton.upenn.edu/pages/support/programming-wrds/programming-python/querying-wrds-data-python/)**.

Load the CBOE S&P 500 Volatility Index (VIX) and store it to `data`. Print the last 10 observations using `.tail(10)` method. 

In [ ]:
data = db.get_table(library='cboe', table='cboe', columns=['date', 'vix'])

In [ ]:
data.tail(10)

Drop NaN values from your DataFrame using `.dropna()`. Plot the CBOE S&P 500 Volatility Index (VIX) using `.plot()` from Pandas. Before doing so, set the index of the Pandas DataFrame to date using `.set_index()`.

In [ ]:
plt.rcParams['figure.figsize'] = [18, 6] #set figure width, height
data = data.dropna()
data.set_index('date').plot()
plt.show()

## 2. Basic statistics in SciPy <a name="basic"></a>


Download a time series of prices for Nvidia, Electronic Arts, and Adobe Systems and save your data to `prices`. Make sure your time series is from 2000-01-01 to today. Use the `db.raw_sql()` function with a SQL query statement. Inspect the head and tail of your DataFrame.


In [ ]:
sql_query =  """
SELECT tic, datadate, prccm 
FROM comp.secm 
WHERE tic IN ('NVDA', 'EA', 'ADBE') AND datadate>='2000-01-01'
"""

prices = db.raw_sql(sql_query)

In [ ]:
prices.head(10)

In [ ]:
prices.tail(10)

Set your index with the date and the ticker and create a pandas multiindex DataFrame. Drop the NaN values and plot the time series by using the `subplots=True` argument. 

In [ ]:
prices = prices.set_index(['tic','datadate'])
prices = prices.dropna()

In [ ]:
plt.rcParams['figure.figsize'] = [16, 10] #set figure width, height
prices.unstack(level=0).plot(subplots=True)
plt.show()

Now calculate the log returns of the NVIDIA stock price time series. Use `np.mean()` and `np.std()` to compute the mean and standard deviation of the return series. 

In [ ]:
NVIDIA = prices.loc["NVDA"]
NVIDIA_log_returns = np.log(NVIDIA) - np.log(NVIDIA.shift(1))
NVIDIA_log_returns = NVIDIA_log_returns.dropna()

In [ ]:
mean = np.mean(NVIDIA_log_returns)
print("Mean of NVIDA returns: " + str(mean))
stdev = np.std(NVIDIA_log_returns)
print("Standard deviation of NVIDA returns: " + str(stdev))

### Testing for Normality
In financial modeling, we often assume a normal distribution for financial returns. In the real world, this is rarely the case. For stocks over short timescales, such as intraday series, the distributions are not assumed to be normal. In fact, they have fatter tails. Over longer periods, the return distribution starts to look normal, but only statistical tests can help to draw conclusions. 

To test for normality, first import the stats module from SciPy using `from scipy import stats`. Then, perform the Shaprio-Wilk test and plot a Histogram of the return series by using `.plot(kind="hist", bins=100)`. Check `help(scipy.stats)` to learn more about the statistical functions and tests in SciPy.

**Shapiro-Wilk test:** The null hypothesis of the test is that the data are normally distributed. If the p-value is less than `0.05`, then the null hypothesis is rejected and there is ''evidence'' that the data do not come from a normally distributed population. Write a `shapiro_wilk_test` function to help us interpret the results.

In [ ]:
from scipy import stats

In [ ]:
def shapiro_wilk_test(alpha, data):
    p_value = stats.shapiro(data)[1]
    if p_value > alpha:
        print("We fail to reject the null hypothesis of the Shaprio-Wilk test with a p value of " +  str(p_value))
    else:
        print("We reject the null hypothesis of the Shaprio-Wilk test with a p value of " + str(p_value))

In [ ]:
shapiro_wilk_test(alpha = 0.05, data = NVIDIA_log_returns)

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5] #set figure width, height
NVIDIA_log_returns.plot(kind="hist", bins=100)
plt.show()

### Correlation of stock returns

In portfolio optimization, diversification among different asset classes, industries and risk levels is a key priority. To test for diversification, the most common and direct measurement is the concept of correlation. 

Calculate the Pearson correlation coefficent between `NVIDIA` and `ADOBE` using `scipy.stats.pearsonr(x,y)` defined by: 

\begin{equation}
r = \frac{{}\sum_{i=1}^{n} (x_i - \overline{x})(y_i - \overline{y})}
{\sqrt{\sum_{i=1}^{n} (x_i - \overline{x})^2(y_i - \overline{y})^2}}
\end{equation}


Like other correlation coefficients, this one varies between -1 and +1, with 0 implying no correlation. Correlations of -1 or +1 imply an exact linear relationship. Positive correlations imply that as x increases, so does y. Negative correlations imply that as x increases, so does y (Source: docs.scipy.org).

In [ ]:
ADBE = prices.loc["ADBE"]
ADBE_log_returns = np.log(ADBE) - np.log(ADBE.shift(1))
ADBE_log_returns = ADBE_log_returns.dropna()

In [ ]:
correlation, p_value = stats.pearsonr(NVIDIA_log_returns.prccm, ADBE_log_returns.prccm)
print("correlation is:" + str(correlation))
print("p-value is: " + str(p_value))

Note that a p-value less than `0.05` indicates stronger evidence that the correlation is different from zero. 

Now visualize both `NVIDIA` and `ADOBE` in a scatter plot.

In [ ]:
plt.rcParams['figure.figsize'] = [10, 6] #set figure width, height
plt.scatter(NVIDIA_log_returns,ADBE_log_returns)
plt.show()

***

### Hands-on

1. Connect to the [Wharton Research Data Services (WRDS)](https://wrds-www.wharton.upenn.edu/).


2. Inspect the Compustat Capital IQ Fundamentals library. For more information, see the documentation of WRDS Compustat Capital IQ.


3. Download quarterly sales data from `Apple` and `Netflix Inc` from `2010-01-01` until today. Clean your dataset and plot your data. What do you see? 


4. Download the stock prices of `IBM`, `MSI` (Motorola Solutions Inc., for those who grew up with flip phones), `INTC`, `NVDA` and `AMD` from `2006-01-01` until today. Each of these are constituents of the S&P500 stock market index. Calculate log returns and clean your return time series. 


5. Besides the Shapiro-Wilk test, there are a few more statistical methods for testing for normality in SciPy. Perform the Kolmogorov-Smirnov test on your return series. This test allows to test if distribution G(x) equals distribution F(x). Under the null hypothesis, the two distributions are identical, G(x)=F(x) (Source: SciPy). To do this, first write a function called `kolmogorov_smirnov_test()` that helps you interpreting your results. Second, write a `for` loop to perform both the Shapiro-Wilk test and the Kolmogorov-Smirnov test and interpret your resutls.
***


## 3. Optimization with SciPy <a name="optimization"></a>


Numpy allows you to achieve fast solutions to complex problems with simple coding. SciPy is built on using Numpy arrays and manipulation to solve common scientific problems such as integration, interpolation, eigenvector decomposition for large sparse matrices, finding minima and/or maxima, and testing distributions.

SciPy comes with an optimization package and allows you to solve almost any kind of minimization problem easily and quickly. A basic example of a minimization problem is given by **simple linear regression**:
\begin{equation}
y_i = \alpha + \beta x_i + \epsilon_i
\end{equation}

The equation describes a line with a slope of $\beta$, an intercept of $\alpha$, and an error term of $\epsilon_i$. $y_i$ is called the dependent variable and $x_i$ is the independent variable. We will learn more about linear regression in the next lesson.

Now let us use SciPy to fit a linear regression model to real data. From WRDS, download the **Fundamentals of Motorola Solutions Inc** where `date <='2010-01-01'`. You are interested in the relationship between **total revenue and cash dividends**. Make sure your index is set to date. Plot the relationship in a scatterplot using matplotlib.

In [ ]:
sql_query ="""
SELECT tic, datadate, REVTQ, DVY
FROM comp.fundq
WHERE tic IN ('MSI') AND datadate<='2010-01-01'
"""

fundamentals = db.raw_sql(sql_query)
fundamentals = fundamentals.set_index(['datadate'])
fundamentals = fundamentals.dropna()

In [ ]:
plt.rcParams['figure.figsize'] = [10, 6] #set figure width, height
plt.scatter(fundamentals.revtq, fundamentals.dvy)
plt.show()

As the scatterplot suggests, there may be a linear relationship between total revenue and cash dividends for this company. Now fit a linear regression model and plot the line in the scatterplot. Use `stats' from the SciPy package to run `lineregress'. 

Make sure that the regression equation is displayed in the scatterplot legend.

In [ ]:
from scipy import stats

In [ ]:
slope, intercept = stats.linregress(fundamentals.revtq,fundamentals.dvy)[:2]
regression_line = slope * fundamentals.revtq + intercept

plt.rcParams['figure.figsize'] = [10, 6] #set figure width, height
plt.plot(fundamentals.revtq, regression_line, 'r', label='y={:.2f}x{:.2f}'.format(slope,intercept))
plt.scatter(fundamentals.revtq,fundamentals.dvy)
plt.legend(fontsize=15)
plt.show()

As you can see in the plot, the regression line fits the data quite well, although we have some data points that are far from the line. So we may want to fit another model, in this case a nonlinear model, which should improve our fit. Note: We will cover OLS and other types of analysis in more detail in future sessions. 

Perform a polynomial fit to the data using Numpy's `polyfit` function. Read the documentation on how to use this function properly. For our purposes, we want to estimate the following model:

\begin{equation}
p(x) = c_0 + c_1x + c_2x^2
\end{equation}

After fitting this model to the data, save the coefficients and write the model to the console. Again, visualize your fit as you did before in the scatterplot.

In [ ]:
x = fundamentals.revtq
y = fundamentals.dvy

In [ ]:
degree = 2
f = np.polyfit(x, y, degree)
p = np.poly1d(f)
quadratic, linear, intercept  = p.coeffs  

In [ ]:
fit = intercept + linear*x + quadratic*x**2

plt.rcParams['figure.figsize'] = [10, 6] #set figure width, height
plt.plot(fundamentals.revtq, fit, 'g', label='y={:.2f} + {:.2f}x + {:.2f}x^2'.format(intercept, linear, quadratic))
plt.scatter(fundamentals.revtq,fundamentals.dvy)
plt.legend(fontsize=15)
plt.show()

***
### Hands-on

1. In the last exercise, you calculated the returns of several stocks. Now you suspect that there might be a linear relationship between `NVDA` and `AMD` since both companies belong to the same industry. Subset the Pandas DataFrame `returns` so that your index starts at the beginning of `2008` and visualize your data in a scatterplot.


2. Calculate the Pearson correlation and the respective p-value. What do the results suggest?


3. Perform a linear regression model, where you regress AMD returns on NVDA returns (`x on y`). Visualize your fit. Save the model and go to the SciPy docs and inspect what the `stats.linregress()` model returns. Did you notice that it returns the Pearson correlation coefficient? And is our slope coefficient statistically significant? What does this tell you?


## 4. Session Takeaways <a name="takeaway"></a>

*What have we learned in this session?*

- Downloading data from WRDS via sql queries and data visualization
- Introduction to the statistical toolbox of Python: SciPy
- The concept of correlation 
- Testing for normality
- Optimization with SciPy: linear regression and polynomial fitting

*What's next?*

Today we have only scratched the surface of SciPy. In fact, SciPy offers many more algorithms, including statistical interpolation, mathematical integration, and numerical solution functions. For more information, see the documentation for the SciPy library. 
***
